In [1]:
! pip install -U spacy -q

In [2]:

!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.8.7                         
Location         /usr/local/lib/python3.12/dist-packages/spacy
Platform         Linux-6.6.105+-x86_64-with-glibc2.35
Python version   3.12.12                       
Pipelines        en_core_web_sm (3.8.0)        



In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [5]:
import json
f = open('/content/annotations.json')
TRAIN_DATA = json.load(f)

In [6]:
TRAIN_DATA

{'classes': ['PERSON',
  'ORGANISATION',
  'DATE',
  'LOCATION',
  'MONEY',
  'DESIGNATION'],
 'annotations': [['', {'entities': []}],
  ['Company Name: NovaTech Solutions Pvt. Ltd.',
   {'entities': [[14, 41, 'ORGANISATION']]}],
  ['Financial Year: 2024-2025', {'entities': [[16, 25, 'DATE']]}],
  ['Report Date: April 15, 2025', {'entities': [[13, 27, 'DATE']]}],
  ['Location: Hyderabad, India', {'entities': [[10, 26, 'LOCATION']]}],
  None,
  None,
  None,
  None,
  ['NovaTech Solutions Pvt. Ltd. is a technology-based firm specializing in cloud computing, AI-driven analytics, and enterprise software solutions. ',
   {'entities': [[0, 18, 'PERSON']]}],
  ['During the fiscal year 2024-2025, the company expanded its operations across three new regions: Bengaluru, Chennai, and Pune.',
   {'entities': [[23, 32, 'DATE'], [96, 125, 'LOCATION']]}],
  None,
  None,
  None,
  None,
  ['Total Revenue: ₹125,48,30,000 (INR 125.48 Crores)',
   {'entities': [[15, 49, 'MONEY']]}],
  ['Revenue from Cl

In [9]:
from tqdm import tqdm

for item in tqdm(TRAIN_DATA['annotations']):
    if item is None or not isinstance(item, (list, tuple)) or len(item) != 2:
        print("Skipping invalid item:", item)
        continue

    text, annot = item
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot.get("entities", []):
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy")


100%|██████████| 78/78 [00:00<00:00, 7218.79it/s]

Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid item: None
Skipping invalid ite

In [10]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [11]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     64.19    0.00    0.00    0.00    0.00
 57     200         97.84   1973.06   98.04   98.04   98.04    0.98
124     400         65.97    220.95   96.08   96.08   96.08    0.96
216     600         59.53    284.42   98.04   98.04   98.04    0.98
316     800         49.73    258.13   98.04   98.04   98.04    0.98
416    1000         91.45    284.51   98.04   98.04   98.04    0.98
593    1200         91.39    494.93   98.04   98.04   98.04    0.98
793    1400         48.22    551.37   98.04   98.04   9

In [12]:
nlp_ner = spacy.load("/content/model-best")

In [13]:
doc = nlp_ner("Market analysts at MorganEast Research, based in Singapore, forecast that the Indian digital lending sector will surpass ₹1.8 lakh crore in transaction volume by 2027, with Aurora capturing approximately 6.4% market share.")

In [14]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter